In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("dataset.csv")
#dataset = tf.data.Dataset.from_tensor_slices(([str(i) for i in df["text"]], df["stars"]))

In [3]:
df = df.dropna()

In [4]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=64)
tokenizer.fit_on_texts([str(i) for i in df["text"]])

In [5]:
def clean_data_to_length(data, length, new_shape):
    ret = []
    for i in data:
        if len(i) > length:
            i = i[:length]
        elif len(i) < length:
            i = i + [0 for i in range(length - len(i))]
        #ret.append(np.reshape(np.array(i), new_shape))
        ret.append(np.array(i))
    return ret

In [6]:
clean_data = clean_data_to_length(tokenizer.texts_to_sequences([str(i) for i in df["text"]]), 64, (1, 8, 8, 1))

In [71]:
model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Conv2D(1, (3, 3), padding='same', input_shape=(8, 8, 1)))
#model.add(tf.keras.layers.Flatten())
#model.add(tf.keras.layers.Dense(1))
model.add(tf.keras.layers.Dense(64, input_shape=(64, 0)))
model.add(tf.keras.layers.Dense(128))
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Dense(5))

In [72]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 64, 64)            64        
_________________________________________________________________
dense_19 (Dense)             (None, 64, 128)           8320      
_________________________________________________________________
dense_20 (Dense)             (None, 64, 256)           33024     
_________________________________________________________________
dense_21 (Dense)             (None, 64, 5)             1285      
Total params: 42,693
Trainable params: 42,693
Non-trainable params: 0
_________________________________________________________________


In [73]:
clean_stars = np.array(list(np.array(i) for i in df["stars"]))
#np.array(list(df["stars"]))

In [74]:
model.compile(loss='mean_squared_error', optimizer='sgd')

In [75]:
model.fit(x=np.array(clean_data), y=clean_stars, batch_size=64, epochs=5, verbose=1, validation_split=.2, validation_freq=1)

ValueError: Error when checking input: expected dense_18_input to have 3 dimensions, but got array with shape (533580, 64)